### 모두를 위한 머신러닝/딥러닝 강의
김성훈 교수님의 모두를 위한 머신러닝/딥러닝 강의 중 lab 강의 코드입니다.
## Lab10_2 Drop out, Batch normalization
강의에서 다룬 Drop out을 mnist data에 활용해보는 코드, 네트워크는 2개의 hidden layer를 가지는 (784, 512, 256, 10)의 구조, 여기에서는 Relu activation function, He initialization을 활용하고, 기존의 stochastic gradient destenct가 아닌 Adam 방법론을 활용함.

### Drop out

In [1]:
import os, sys
import tensorflow as tf
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777) # for reproducibility
print(os.getcwd(),'\n', os.listdir())

D:\dev 
 ['.ipynb_checkpoints', 'Data structures and algorithms_warm_up.ipynb', 'Data structures and algorithms_week_1.ipynb', 'DLEL', 'DLEL.zip', 'DLEL_10_1_accuracy_loss.PNG', 'DLEL_10_1_activation_histogram.PNG', 'DLEL_10_1_example_graph.PNG', 'DLEL_10_1_Weight_initialization.ipynb', 'DLEL_10_2_Drop-out_&_Batch_normalization.ipynb', 'logs', 'MNIST_data', 'py-automate', 'untitled.txt']


In [2]:
## load data and parameter
mnist = input_data.read_data_sets('./MNIST_data/', one_hot = True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# define He initializer
def he_init(n_input, n_output):
    stddev = tf.sqrt(2.0 / (n_input))
    return tf.truncated_normal_initializer(stddev = stddev)

In [4]:
# graph setting
X = tf.placeholder(dtype = tf.float32, shape = [None, 784], name = 'Xinput')
Y = tf.placeholder(dtype = tf.float32, shape = [None, 10], name = 'Yinput')

# dropout (keep-prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(dtype = tf.float32)

with tf.variable_scope('layer1'):
    w1 = tf.get_variable(name = 'w1', shape = [784, 512], initializer = he_init(784, 512))
    b1 = tf.Variable(initial_value = tf.random_normal(shape = [512]), name = 'b1')
    layer1 = tf.nn.relu(tf.matmul(X, w1) + b1)
    
with tf.variable_scope('layer2'):
    w2 = tf.get_variable(name = 'w2', shape = [512, 256], initializer = he_init(512, 256))
    b2 = tf.Variable(initial_value = tf.random_normal(shape = [256]), name = 'b2')
    layer2 = tf.nn.relu(tf.matmul(layer1, w2) + b2)
    layer2 = tf.nn.dropout(x = layer2, keep_prob = keep_prob, name = 'drop-out')

with tf.variable_scope('layer3'):
    w3 = tf.get_variable(name = 'w3', shape = [256, 10], initializer = he_init(256, 10))
    b3 = tf.Variable(initial_value = tf.random_normal(shape = [10]), name = 'b3')
    score = tf.matmul(layer2, w3) + b3

with tf.variable_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = score, labels = Y))
    loss_scalar = tf.summary.scalar(name = 'loss', tensor = loss)

with tf.variable_scope('train'):
    train = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)
    
with tf.variable_scope('accuracy'):
    is_correct = tf.equal(tf.argmax(score, 1), tf.argmax(Y, axis = 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, dtype = tf.float32))
    accuracy_scalar = tf.summary.scalar(name = 'accuracy', tensor = accuracy)

In [5]:
# parameter 
batch_size = 100
epochs = 100

with tf.Session() as sess:
    
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs/dropout')
    writer.add_graph(sess.graph)
    
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for epoch in range(epochs):
        
        for step in range(total_batch):
            
            tr_batch_xs, tr_batch_ys = mnist.train.next_batch(batch_size = batch_size)
            val_batch_xs, val_batch_ys = mnist.validation.next_batch(batch_size = mnist.validation.num_examples)
            summary, _, loss_tr = sess.run([merged_summary, train, loss], feed_dict = {X : tr_batch_xs, Y : tr_batch_ys, keep_prob : 0.5})
            loss_val = sess.run(loss, feed_dict = {X : val_batch_xs, Y : val_batch_ys, keep_prob : 1})
                
        if epoch % 5 == 0 or epoch == (epochs - 1):
            print('epoch : {:4}, tr_loss : {:.3f}, val_loss : {:.3f}'.format(epoch, loss_tr, loss_val))
        writer.add_summary(summary, global_step = epoch)
    
    print(sess.run(accuracy, feed_dict = {X : val_batch_xs, Y : val_batch_ys, keep_prob : 1}))
    print(sess.run(accuracy, feed_dict = {X : mnist.test.images, Y : mnist.test.labels, keep_prob : 1}))

epoch :    0, tr_loss : 0.257, val_loss : 0.158
epoch :    5, tr_loss : 0.015, val_loss : 0.061
epoch :   10, tr_loss : 0.067, val_loss : 0.082
epoch :   15, tr_loss : 0.020, val_loss : 0.083
epoch :   20, tr_loss : 0.044, val_loss : 0.103
epoch :   25, tr_loss : 0.001, val_loss : 0.111
epoch :   30, tr_loss : 0.006, val_loss : 0.122
epoch :   35, tr_loss : 0.054, val_loss : 0.128
epoch :   40, tr_loss : 0.014, val_loss : 0.139
epoch :   45, tr_loss : 0.026, val_loss : 0.113
epoch :   50, tr_loss : 0.000, val_loss : 0.114
epoch :   55, tr_loss : 0.032, val_loss : 0.115
epoch :   60, tr_loss : 0.000, val_loss : 0.124
epoch :   65, tr_loss : 0.001, val_loss : 0.168
epoch :   70, tr_loss : 0.000, val_loss : 0.158
epoch :   75, tr_loss : 0.000, val_loss : 0.185
epoch :   80, tr_loss : 0.000, val_loss : 0.155
epoch :   85, tr_loss : 0.000, val_loss : 0.165
epoch :   90, tr_loss : 0.000, val_loss : 0.193
epoch :   95, tr_loss : 0.000, val_loss : 0.201
epoch :   99, tr_loss : 0.000, val_loss 